In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from data_loader import load_track_data, load_race_data
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import math

from data_loader import transform_coordinates


In [23]:
# Plot the reduced track in 2D using matplotlib
track_left, track_right = load_track_data(distance_pr_dot = 1)

track_left["index"] = track_left["index"].astype(str)
track_right["index"] = track_right["index"].astype(str)
track_left

c:\GitHub\RacingTrajectories\data_loader.py:6: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

c:\GitHub\RacingTrajectories\data_loader.py:7: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.



,index,Timestamp,Y-Coords,Z-Coords,X-Coords
0,0.0,12.072,2.187290,63.983009,764.897095
140,140.0,14.259,2.231630,63.973942,763.882812
167,167.0,14.685,2.268255,63.964184,762.850464
188,188.0,0.192,2.300150,63.956116,761.810608
205,205.0,0.456,2.328267,63.948250,760.798889
...,...,...,...,...,...
27248,27248.0,1.818,2.557289,63.860271,742.016357
27255,27255.0,1.929,2.577993,63.858112,740.951294
27263,27263.0,2.052,2.597204,63.851013,739.898376
27272,27272.0,2.193,2.613663,63.841419,738.897644


In [33]:

fig = go.Figure()
fig.add_trace(go.Scatter(
        x=track_left['X-Coords'], 
        y=track_left['Y-Coords'], 
        mode='lines+markers',  # Change to 'lines+markers' to show dots
        name='Track Left', 
        line=dict(width=0.8),
        marker=dict(size = 0.8),
        text='Index: ' + track_left['index'],  # Add index information for hover
        hoverinfo='text+x+y'  # Customize hover information
    )
)

fig.add_trace(go.Scatter(
        x=track_right['X-Coords'], 
        y=track_right['Y-Coords'], 
        mode='lines+markers',  # Change to 'lines+markers' to show dots
        name='Track Right', 
        line=dict(width=0.8),
        marker=dict(size = 0.8),
        text='Index: ' + track_right['index'],  # Add index information for hover
        hoverinfo='text+x+y'  # Customize hover information
    ))
fig.update_layout(title='Track Comparison', xaxis_title='X-Axis', yaxis_title='Y-Axis')

# change size of the plot
fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
)

default_zoom_x = [-850, 1100]  # Replace these with your desired x-axis range
default_zoom_y = [-300, 450]  # Replace these with your desired y-axis range 

fig.update_xaxes(range=default_zoom_x)
fig.update_yaxes(range=default_zoom_y)

fig.show()


In [25]:
# Import real left and right track data. from ./Data/Map_details/Left_boundary.csv and Right_boundary.csv
### NOTE ONLY RUN ONCE

for txt_file in ['Left_boundary', 'Right_boundary']:
    with open(f'./Data/Map_details/{txt_file}.txt', 'r') as f:
        # left_boundary = pd.read_csv(f)
        boundary = f.readlines()
        boundary = [x.split() for x in boundary][0]
        boundary = [x[:-2] for x in boundary]
        boundary = [x.split(',') for x in boundary]

        df_boundary = pd.DataFrame(boundary, columns = ['Longitude', 'Latitude'])
        # save to csv file
        df_boundary.to_csv(f'./Data/Map_details/{txt_file}.csv', index = False)


In [26]:
real_track_left_side  = pd.read_csv('./Data/Map_details/Left_boundary.csv')
real_track_right_side = pd.read_csv('./Data/Map_details/Right_boundary.csv')
real_track_right_side

,Longitude,Latitude
0,6.947242,50.335378
1,6.947557,50.335586
2,6.947965,50.335849
3,6.948183,50.335997
4,6.948543,50.336224
...,...,...
388,6.943929,50.333223
389,6.944191,50.333396
390,6.944691,50.333727
391,6.947055,50.335270


## NOTE FOR TRANSFORMATION : 

The transformation process was as followed. 
1. Try to allign the 2 u-turns above the center piece.
2. Allign the turn which is directly below the center piece, as the vertical allignment should 
3. Re-allign the 2 u-turns above the center piece, as the horizontal allignment should be correct now.
4. Check the error of rotation of the top right part, and left bottom part of the track, and rotate accordingly 
5. Re-allign the 2 u-turns above the center piece, as the horizontal allignment should be correct now.
6. Check the error of rotation of the top right part, and left bottom part of the track, and rotate accordingly (smaller)
7. Re-allign the 2 u-turns above the center piece, as the horizontal allignment should be correct now.
8. Confirm that the track rotation is correct, and the scale is almost correct.
9. find the left most part of the track, and the right most part of the track. Allign by the x-axis with shifts and scale.
10. find the top most part of the track, and the bottom most part of the track. Allign by the y-axis with shifts and scale.

This process seemed to result in the most accurate transformation of the track, although it is not perfect.

In [27]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula 
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


def calc_track_length(track_df):
    """
    Calculate the length of the track in km
    """
    length = 0
    try:
        latitude = track_df['Latitude'].to_list()
        longitude = track_df['Longitude'].to_list()
    except KeyError:
        latitude = track_df['Y-Coords'].to_list()
        longitude = track_df['X-Coords'].to_list()

    for i in range(len(longitude)-1):
        length += haversine_distance(latitude[i], longitude[i], latitude[i+1], longitude[i+1])
    return length


rotation_angle = 45.15 # More is left, less is to the right
scale_x = 0.00001406
scale_y = 0.000009005
shift_x = 6.940315
shift_y = 50.330715
center_x = 0
center_y = 0


In [28]:

def plot_track(track_left, track_right, real_track_left_side, real_track_right_side):
    # plot the track, but use plotly for interactivity
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=track_left['X-Coords'], y=track_left['Y-Coords'], mode='lines', name='Track Left', line=dict(width=0.8)))
    fig.add_trace(go.Scatter(x=track_right['X-Coords'], y=track_right['Y-Coords'], mode='lines', name='Track Right', line=dict(width=0.8)))
    fig.add_trace(go.Scatter(x=real_track_left_side['Longitude'], y=real_track_left_side['Latitude'], mode='lines', name='Real Track Left', line=dict(width=0.8)))
    fig.add_trace(go.Scatter(x=real_track_right_side['Longitude'], y=real_track_right_side['Latitude'], mode='lines', name='Real Track Right', line=dict(width=0.8)))
    fig.update_layout(title='Track Comparison', xaxis_title='Longitude', yaxis_title='Latitude')

    # plot a center point for the track as the shift_x, shift_y
    shift_x = 6.940328
    shift_y = 50.3307159
    fig.add_trace(go.Scatter(x=[shift_x], y=[shift_y], mode='markers', name='Center Point'))
    
    # change size of the plot
    fig.update_layout(
        autosize=False,
        width=1000,
        height=1000,
    )

    #ALL
    default_zoom_x = [6.933, 6.951]  # Replace these with your desired x-axis range
    default_zoom_y = [50.324, 50.338]  # Replace these with your desired y-axis range

    # # Bottom Left
    # default_zoom_x = [6.9339, 6.9357]  # Replace these with your desired x-axis range
    # default_zoom_y = [50.3238, 50.3252]  # Replace 

    # Top Right
    # default_zoom_x = [6.9479, 6.951]  # Replace these with your desired x-axis range
    # default_zoom_y = [50.3355, 50.338]  # Replace 

    # Mid
    default_zoom_x = [6.938, 6.942]  # Replace these with your desired x-axis range
    default_zoom_y = [50.3305, 50.334]  # Replace 

    fig.update_xaxes(range=default_zoom_x)
    fig.update_yaxes(range=default_zoom_y)
    
    fig.show()


rotation_angle = 45.15 # More is left, less is to the right
scale_x = 0.00001406
scale_y = 0.000009005
shift_x = 6.940315
shift_y = 50.330715
center_x = 0
center_y = 0



trans_track_left = transform_coordinates(track_left_side,rotation_angle, scale_x  ,scale_y  ,  shift_x  ,  shift_y  , center_x ,center_y )
trans_track_right = transform_coordinates(track_right_side,rotation_angle,scale_x  ,scale_y  ,  shift_x  ,  shift_y  , center_x ,center_y )

trans_track_left = transform_coordinates(track_left_side,rotation_angle, scale_x  ,scale_y  ,  shift_x  ,  shift_y  , center_x ,center_y )
trans_track_right = transform_coordinates(track_right_side,rotation_angle,scale_x  ,scale_y  ,  shift_x  ,  shift_y  , center_x ,center_y )


print("Transformed Left Track Length: ",calc_track_length(trans_track_left))
print("Real Left Track Length:        ",calc_track_length(real_track_left_side))
print("")
print("Transformed Left Track Right: ",calc_track_length(trans_track_right))
print("Real Right Track Length:      ",calc_track_length(real_track_right_side))


plot_track(track_left = trans_track_left, track_right = trans_track_right, 
            real_track_left_side = real_track_left_side, 
            real_track_right_side = real_track_right_side)

Transformed Left Track Length:  5.206618316709292
Real Left Track Length:         5.187256161031783

Transformed Left Track Right:  5.111147588385996
Real Right Track Length:       5.085862132061388
